# Thinking 1 ：在实际工作中，FM和MF哪个应用的更多，为什么

答：
### MF 矩阵分解：
>对稀疏的部分进行填补，通常情况下是在 User - Item 两个特征时使用，缺点是仅考虑了用户的行为，未考虑用户自身的特征以及商品自身的特征等；

### FM 因子分解机：
>选取多个特征，并在线性回归的基础上加入了交互项（二阶，甚至多阶）进行分析，与矩阵分解算法相比较可以得到更加准确的结论；  
但对于交互项中稀疏的部分无法拟合系数，采用矩阵分解的方法，填补缺失的系数，得到更准确的模型。

### 两者的不同：
1. FM 考虑了更多维度的特征，而MF 仅考虑了 User 与 Item 两个维度；
2. FM 不仅考虑了交叉项部分（二阶，高阶），还考虑了线性部分。

# Thinking 2 ：FFM与FM有哪些区别？

答：  
FM 每个特征只有一个隐向量表示；
FFM 中每个特征有几个不同的隐向量。

# Thinking 3 ： DeepFM相比于FM解决了哪些问题，原理是怎样的

答：  
FM 不仅可以考虑低阶特征，而且可以考虑高阶的特征，但考虑高阶特征时，计算量较大，因此考虑使用 DNN 提取高阶特征。

# Thinking 4 ：Surprise工具中的baseline算法原理是怎样的？BaselineOnly和KNNBaseline有什么区别？

答：  
通过设置所有商品平均值，用户对整体偏差，商品对整体偏差的方式对商品进行打分。  
KNNBaseline是在KNNWithMeans的基础上，用baseline代替均值。

# Thinking 5 ：基于邻域的协同过滤都有哪些算法，请简述原理

答：  
KNNBasic、KNNWithMeans  
以相似度选取的领域，落在以当前点为中心，距离为 K 的区域中的所有点都作为邻居。  

# Action 1 ：使用libfm工具对movielens进行评分预测，采用SGD优化算法

```
libFM -task r -train ratings.dat.libfm -test ratings.dat.libfm -dim '1,1,8' -method sgd -learn_rate 0.01 -regular ’0,0,0.01’ -init_stdev 0.1 -out sgdout.txt
```
#Iter= 80       Train=0.77122   Test=0.77122    
#Iter= 81       Train=0.771172  Test=0.771172    
#Iter= 82       Train=0.771125  Test=0.771125    
#Iter= 83       Train=0.771079  Test=0.771079    
#Iter= 84       Train=0.771035  Test=0.771035  
#Iter= 85       Train=0.770992  Test=0.770992  
#Iter= 86       Train=0.77095   Test=0.77095  
#Iter= 87       Train=0.770909  Test=0.770909  
#Iter= 88       Train=0.770869  Test=0.770869  
#Iter= 89       Train=0.77083   Test=0.77083  
#Iter= 90       Train=0.770792  Test=0.770792  
#Iter= 91       Train=0.770755  Test=0.770755  
#Iter= 92       Train=0.770719  Test=0.770719  
#Iter= 93       Train=0.770684  Test=0.770684  
#Iter= 94       Train=0.77065   Test=0.77065  
#Iter= 95       Train=0.770616  Test=0.770616  
#Iter= 96       Train=0.770584  Test=0.770584  
#Iter= 97       Train=0.770552  Test=0.770552  
#Iter= 98       Train=0.770521  Test=0.770521  
#Iter= 99       Train=0.77049   Test=0.77049  
Final   Train=0.77049   Test=0.77049  

# Action 2 ： 使用DeepFM对movielens进行评分预测

## 使用google colab训练：

```python
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, get_feature_names

#数据加载
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/")

data = pd.read_csv("movielens_sample.txt")
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ['rating']

# 对特征标签进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])
# 计算每个特征中的 不同特征值的个数
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
print(fixlen_feature_columns)
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'], )
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=1, verbose=True, validation_split=0.2, )
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)
```

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "  
1/1 [==============================] - 0s 305ms/step - loss: 14.4452 - mse: 14.4452 - val_loss: 11.5209 - val_mse: 11.5209  
test RMSE 3.879871131880542  

# Action 3 ：使用基于邻域的协同过滤（KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline中的任意一种）对MovieLens数据集进行协同过滤，采用k折交叉验证(k=3)，输出每次计算的RMSE, MAE

```python
from surprise import KNNWithMeans
from surprise import Dataset, Reader
import surprise
from surprise import accuracy
from surprise.model_selection import KFold

# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('./ratings.csv', reader=reader)


# ItemCF 计算得分
# 取最相似的用户计算时，只取最相似的k个
algo = KNNWithMeans(k=50, sim_options={'user_based': False, 'verbose': 'True'})


# 定义K折交叉验证迭代器，K=3
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)
```

Computing the msd similarity matrix...  
Done computing similarity matrix.  
RMSE: 0.8552  
Computing the msd similarity matrix...  
Done computing similarity matrix.  
RMSE: 0.8585  
Computing the msd similarity matrix...  
Done computing similarity matrix.  
RMSE: 0.8553  

# 本章任务

In [1]:
import xlrd
data = xlrd.open_workbook('L5自测文档.xls')
#通过索引顺序获取
table = data.sheet_by_index(0)

""" 工作表中行/列的操作 """
#获取该sheet中的有效行数
nrows = table.nrows  
print(nrows)
row_index, col_index = 0, 0
# 获取某行信息
for row_index in range(2, nrows-9):
    print(table.row(row_index)[:2])
for row_index in range(nrows-8, nrows):
    print(table.row(row_index)[:2], table.row(row_index)[-2])

23
[text:'原理', text:'FM算法的原理']
[text:'原理', text:'FM算法的计算复杂度']
[text:'原理', text:'FFM算法的原理']
[text:'原理', text:'DeepFM算法的原理']
[text:'工具', text:'libfm']
[text:'工具', text:'libffm']
[text:'工具', text:'xlearn']
[text:'工具', text:'deepctr']
[text:'原理', text:'基于邻域的协同过滤 UserCF，ItemCF']
[text:'原理', text:'相似邻居的定义']
[text:'原理', text:'相似度计算的方式']
[text:'工具', text:'Surprise工具中的基于邻域的协同过滤算法使用']
[text:'Thinking1', text:'在实际工作中，FM和MF哪个应用的更多，为什么'] text:'1、能简要说明FM和MF的区别（5point）\n2、能简要说明FM在推荐系统，以及应用场景中的作用（5point）\n'
[text:'Thinking2', text:'FFM与FM有哪些区别？'] text:'1、能简要说明区别（10point）\n'
[text:'Thinking3', text:'DeepFM相比于FM解决了哪些问题，原理是怎样的'] text:'1、能说明DeepFM相比于FM有哪些改进的地方（5points）\n2、能说明DeepFM的原理，FM+DNN模型（5points）'
[text:'Thinking4', text:'Surprise工具中的baseline算法原理是怎样的？BaselineOnly和KNNBaseline有什么区别？'] text:'1、能简要说明baseline的原理（5points)\n2、能简要说明BaselineOnly和KNNBaseline的区别（5points)'
[text:'Thinking5', text:'基于邻域的协同过滤都有哪些算法，请简述原理'] text:'1、能说出两种不同的基于邻域的协同过滤的算法（5points）\n2、这些算法之间的区别和作用（5pionts）'
[text:'Action1', text:'使用li